In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,InputLayer,Normalization,Conv2D,MaxPool2D,Input
from tensorflow.keras.optimizers import Adam

dataset = tfds.load("malaria",as_supervised=True,data_dir="D:\Dataset")


ds_lenght = len(dataset['train'])
print(f"dataset lenght {ds_lenght}")


TrainRatio = 0.8
TestRatio = 0.1
EvalRatio = 0.1



def resize_and_normalize_img(image, label):
    image = tf.image.resize(image, [224, 224])  
    image = tf.cast(image, tf.float32) / 255.
    return image, label



dataset['train'] = dataset['train'].map(resize_and_normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)


# Splitta il dataset
train_ds = dataset['train'].take(int(ds_lenght * TrainRatio)).batch(4)
remaining_ds = dataset['train'].skip(int(ds_lenght * TrainRatio))
test_ds = remaining_ds.take(int(ds_lenght * TestRatio)).batch(4)
eval_ds = remaining_ds.skip(int(ds_lenght * TestRatio)).batch(4)
print(train_ds)

print(f"train_ds {len(train_ds)}")
print(f"test_ds {len(test_ds)}")
print(f"eval_ds {len(eval_ds)}")

c:\Users\ottav\anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dataset lenght 27558
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
train_ds 5512
test_ds 689
eval_ds 690


In [2]:
import tensorflow as tf

# Ottieni la lista delle GPU disponibili
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
                tf.config.experimental.set_visible_devices(gpu, 'GPU')
                print("GPU settata: ", gpu)
    except RuntimeError as e:
        print(e)

# Definisci il tuo modello con le Functional API
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = tf.keras.layers.Conv2D(filters=5, kernel_size=8, activation="relu")(inputs)
x = tf.keras.layers.MaxPool2D((8,8),(1,1))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=5, kernel_size=4, activation="relu")(x)
x = tf.keras.layers.MaxPool2D((4,4),(1,1))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=5, kernel_size=2, activation="relu")(x)
x = tf.keras.layers.MaxPool2D((2,2),(1,1))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
outputs = tf.keras.layers.Dense(2, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'],
              )

# Addestra il tuo modello
history = model.fit(train_ds, epochs=10, batch_size=4)
model.save('modello.h5')

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.title('Loss Function')
plt.legend()

# Crea un grafico per l'accuratezza
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.title('Accuracy')
plt.legend()

plt.show()


GPU settata:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Epoch 1/10


In [41]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Carica il modello addestrato
model = load_model('modello.h5')

def predict_image(img_path):
    # Carica l'immagine come immagine PIL in formato RGB
    img = image.load_img(img_path, target_size=(224, 224), color_mode="rgb")
    
    # Converti l'immagine in un array numpy e ridimensionala per il modello
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # ricorda di normalizzare come hai fatto durante l'addestramento
    
    # Effettua la previsione
    prediction = model.predict(img_array)
    
    # Stampa il valore predetto
    print("Valore predetto: ", prediction)
    
    # Stampa la previsione
    if prediction[0][0] > prediction[0][1]:
        print("La previsione per l'immagine è: Vero")
    else:
        print("La previsione per l'immagine è: Falso")

# Chiama la funzione predict_image
predict_image(r"c:\Users\ottav\Desktop\C124P85ThinF_IMG_20151002_154846_cell_139.png")


1/1 [==============================] - 0s 140ms/step
Valore predetto:  [[0.45782357 0.58897835]]
La previsione per l'immagine è: Falso
